# QKD Image Encryption Project

This notebook implements both BB84 and E91 key distribution protocols and image encryption.

In [3]:
%pip install qiskit
%pip install --upgrade typing_extensions
%pip install qiskit-aer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.4 MB 6.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## 1. Imports & Setup

In [4]:

import secrets
import pathlib
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Simulator backend
backend = AerSimulator()
rng = secrets.SystemRandom()
    

## 2. User Parameters

In [6]:

# User tweakables
IMG_PATH        = pathlib.Path('download.png')
BLOCK_SIZE      = 5
ABORT_THRESHOLD = 0.11
SAMPLE_RATE     = 0.10

if not IMG_PATH.exists():
    raise FileNotFoundError(f'Missing {IMG_PATH}')
img_bytes  = IMG_PATH.read_bytes()
n_img_bits = len(img_bytes) * 8
print(f'Image size: {len(img_bytes):,} bytes ({n_img_bits:,} bits)')
    

Image size: 3,844 bytes (30,752 bits)


## 3. Utility Functions

In [7]:

def random_bits(n):
    return (np.random.randint(2, size=n, dtype=np.uint8),
            np.random.randint(2, size=n, dtype=np.uint8))

def hamming(a, b):
    return np.count_nonzero(a != b)

def bits_to_bytes(bits):
    out = bytearray()
    for i in range(0, len(bits), 8):
        byte = 0
        for j in range(8):
            if i + j < len(bits):
                byte = (byte << 1) | int(bits[i + j])
        out.append(byte)
    return bytes(out)
    

## 4. E91 Emitter Circuit

In [8]:

def emitter_circuit(source_bits, source_bases): 
    size = len(source_bases)
    qc = QuantumCircuit(2*size, 2*size, name='Emitter')
    for i, (bit, basis) in enumerate(zip(source_bits, source_bases)):
        qc.x(i); qc.h(i); qc.x(i+size); qc.cx(i, i+size)
        if bit: qc.x(i); qc.x(i+size)
        if basis: qc.h(i); qc.h(i+size)
    return qc
    

## 5. Measurement Circuit

In [9]:

def measure_circuit(alice_bases, bob_bases): 
    size = len(alice_bases)
    qc = QuantumCircuit(2*size, 2*size, name='Measure')
    for i in range(size):
        if alice_bases[i]: qc.h(i)
        if bob_bases[i]: qc.h(i+size)
    return qc
    

## 6. Full Protocol Execution

In [ ]:

def full_quantum_circuit(source_bits, source_bases, alice_bases, bob_bases):
    ec = emitter_circuit(source_bits, source_bases)
    mc = measure_circuit(alice_bases, bob_bases)
    return ec.compose(mc)

def exec_protocol():
    alice_key, bob_key = [], []
    rounds = 0
    while len(alice_key) < n_img_bits:
        rounds += 1
        bits_S, bases_S = random_bits(BLOCK_SIZE)
        bases_A, bases_B = random_bits(BLOCK_SIZE)
        qc = full_quantum_circuit(bits_S, bases_S, bases_A, bases_B)
        qc.measure(range(2*BLOCK_SIZE), range(2*BLOCK_SIZE))
        compiled = transpile(qc, backend)
        result = backend.run(compiled, shots=1).result()
        bitstr = next(iter(result.get_counts()))
        meas = np.fromiter(map(int, bitstr[::-1]), dtype=np.uint8)
        meas_A = meas[:BLOCK_SIZE]; meas_B = meas[BLOCK_SIZE:]
        keep = np.array(bases_A) == np.array(bases_B)
        alice_key.extend(meas_A[keep].tolist())
        bob_key.extend((1 ^ meas_B[keep]).tolist())
        print(f'Round {rounds}: {len(alice_key):,}/{n_img_bits} key bits', end='\r')
    alice_key = np.array(alice_key[:n_img_bits], dtype=np.uint8)
    key_bytes = bits_to_bytes(alice_key)
    cipher = bytes(m ^ k for m, k in zip(img_bytes, key_bytes))
    pathlib.Path('download.enc').write_bytes(cipher)
    dec = bytes(c ^ k for c, k in zip(cipher, key_bytes))
    pathlib.Path('download_decrypted.png').write_bytes(dec)
    assert dec == img_bytes
    print('\n✅ Completed.') 

exec_protocol()
    